# 自动站数据的类


In [2]:
%%time
import pymssql 
import pandas as pd

## sqlserver
server = "172.21.158.201"    # 连接服务器地址
user = "down"# 连接帐号
password = "downx"# 连接密码
conn = pymssql.connect(server, user, password, "ZJSZDZDB")  #获取连接

sql_location = "select b.StationName,b.county,b.Town,b.lat,b.Height,b.lon,b.IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR from TAB_Mws2022 as a left join TAB_StationInfo as b on a.IIiii=b.IIiii where\
(b.IIiii in (select IIiii from TAB_StationInfo where(City = '台州') and tTime between '2022-01-25 20:00' and '2022-02-10 20:00'))"


df_location = pd.read_sql(sql_location , con=conn)
df_location

CPU times: user 1.49 s, sys: 90 ms, total: 1.58 s
Wall time: 5.57 s


,StationName,county,Town,lat,Height,lon,IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR
0,街头雪深,天台县,街头镇,29.12444,01140,120.7933,K8734,2022-01-25 23:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
1,后岸,天台县,街头镇,29.07861,01010,120.8139,K8748,2022-01-25 23:00:00,16,42,73,77,73,-9999,4
2,欢岙,天台县,坦头镇,29.18444,02910,121.1128,K8773,2022-01-25 23:00:00,180,18,59,60,59,-9999,5
3,济溪,天台县,街头镇,29.05389,01990,120.8553,K8790,2022-01-25 23:00:00,-9999,-9999,72,73,71,-9999,6
4,叶家村,三门县,海润街道,29.06750,00520,121.4881,K8812,2022-01-25 23:00:00,315,19,75,79,75,-9999,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176568,溪港,仙居县,溪港乡,28.57361,02200,120.3961,K8912,2022-02-10 19:00:00,0,2,81,86,81,-9999,0
176569,四小,仙居县,安洲街道,28.87278,00760,120.7306,K8927,2022-02-10 19:00:00,95,16,80,85,80,-9999,0
176570,山岙雪深,仙居县,横溪镇,28.77944,05540,120.4283,K8932,2022-02-10 19:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
176571,船山村,仙居县,南峰街道,28.81833,01860,120.7011,K8973,2022-02-10 19:00:00,68,43,72,75,72,-9999,0


In [3]:
df_location.to_csv('data_zdz_height.csv')

In [4]:
%%time
import gevent
from math import isnan
import numpy as np
#import modin.pandas as pd 
import pandas as pd
import time
from func import *
import matplotlib
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import json
import math

import pymssql 
import datetime


class zdz_data:
    def __init__(self, start,end):
        self.start = start
        self.end = end
        # 基础数据
        self.station_data = None
        self.day_list = None
        self.rain_line = None
        self.rain_scatter = None
        self.station_all = self.read_csv()
        self.event_data()
        self.rain_data()
        
        
    def sql_date(self):
        '''数据库读取sql数据'''
        print('读取数据库数据')
    def read_csv(self):
        '''
        1.数据读取---此方法为测试方法
        2.数据分段---每天
        3.每天统计
        '''
#         station_Mws = pd.read_csv("Mws_15.csv")
#         station_Aws = pd.read_csv("Aws_15.csv")
#         station_all = pd.concat([station_Aws, station_Mws])
        station_all = pd.read_csv("data_zdz_height.csv")
        # station_all = pd.read_csv("static/data/data_zdz_height.csv")
        return station_all
    def event_data(self):
        '''
        1.数据库查询
        2.数据分段---每天
        3.每天统计
        ['2022-04-14', 200,'高温 浓雾', '降水 大风', 200],
        '''
        dateList = [ ] 
        # 多少天
        dates = self.return_daylist()
        for i in dates:
            #解析每天的数据
            # grouped_county = self.station_all.groupby('county')
            # grouped_IIiii = self.station_all.groupby('IIiii')
            # for i in self.grouped_county.size().index: 
            s_date = i + ' ' + '00:00'
            e_date = i + ' ' + '23:00'
            df_preday = self.station_all[(self.station_all['tTime'] >= s_date) & (self.station_all['tTime'] <= e_date)]              
#             df_RR= df_preday[df_preday['RR'] == -9999 ].count()
#             grouped_county = 
            date_preday = []
            # 日期
            day_date = str(i)
            # 面雨量
            rainfall = 200
            # 气温
            temp_min = df_preday[(df_preday['Height']<6000)&(df_preday['T'] > -400)]['T'].min()
            temp_min = temp_min/10.0
            temp_max = df_preday[(df_preday['Height']<6000)&(df_preday['T'] > -400)]['T'].max()
            temp_max = temp_max/10.0
            temp_label = ''
            if temp_min < 3.0:
                temp_label = '低温'
            elif temp_max > 35.0:
                temp_label = '高温'
            else:
                temp_label = ' '
            # 能见度
            # df_VV= df_preday[(df_preday['VV']>0.1)&(df_preday['VV']<500)]['VV'].count()
            vv = df_preday[(df_preday['VV']>0.1)&(df_preday['VV']<500)]['VV'].count()
            if vv>0:
                view_label = '浓雾'
            else:
                view_label = ' '    
            # 降水
            df_RR= df_preday[(df_preday['RR']>0.3)&(df_preday['RR']<8888)]['RR'].count()
            
            if df_RR > 0:
                pre_label = '降水'
            else:
                pre_label = ' '
            # 大风
            wind_count = df_preday[df_preday['fFy']>187]['fFy'].count()
            if wind_count>0:
                wind_label = '大风'
            else:
                wind_label = ' '
            date_preday = [day_date,rainfall,temp_label + ' '+ view_label, pre_label +' '+ wind_label ,rainfall]
            dateList.append(date_preday)
        self.day_list = dateList
#         print('event_data:返回一个每天的数组和日历所需的数据','气温',dateList)
    def return_daylist(self):
        '''
        返回每天的起始列表
        '''
        dates = []
#         dt = datetime.datetime.strptime(self.start, "%Y-%m-%d")
        dt = datetime.datetime.strptime(self.start[0:10], "%Y-%m-%d")
        date = self.start[:10]
        while date <= self.end[:10]:
            dates.append(date)
            dt = dt + datetime.timedelta(1)
            date = dt.strftime("%Y-%m-%d")
        return dates
    def rain_data(self):
        '''
        1.根据起始时间计算面雨量的sql语句
        2.根据初始语句返回每站的总降水量
        '''
        station_all = self.station_all
        
        data_rain = {
            'rain_sum':{
                'time':[],
                'data':[]
            },
            'IIiii_data':{}
            
        }
        
        grouped_tTime = station_all.groupby('tTime')
        for i in grouped_tTime.size().index:  
            data= grouped_tTime.get_group(i)
            data['RR'].replace(-9999,np.nan,inplace=True)
            rain_mean = data['RR'].mean()/10.0
            rain_time = i
            data_rain['rain_sum']['time'].append(rain_time)
            data_rain['rain_sum']['data'].append(rain_mean)
        # 导出单站数据
        grouped_IIiii = station_all.groupby('IIiii')
        rain_scatter = []
#         {
#          name: "K8515", value: [121.2, 28.6, 110],
#          symbol: 'circle'
#         }
        for i in grouped_IIiii.size().index:  
            data= grouped_IIiii.get_group(i)
            data['RR'].replace(-9999,np.nan,inplace=True)
            station_name = str(i) 
            data_rain['IIiii_data'][station_name] = data
            single_data = {}
            single_data['name'] = station_name
            single_data['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['RR'].sum()/10.0]
            single_data['symble'] = 'circle'
            rain_scatter.append(single_data)
        self.station_data = data_rain['IIiii_data']
        self.rain_line =  [data_rain['rain_sum']['time'],data_rain['rain_sum']['data']]
        self.rain_scatter = rain_scatter  
    def wind_data(self):
        '''
        1.根据sql语句计算8及以上大风的分布和排序
        ''' 
        data_wind_list = []
        station_all = self.station_all
        sort_data = {
            'IIiii':[],
            'county':[],
            'town':[],
            'value':[]
        }
        #wind_max = station_all['fFy'].idxmax(axis=0)
        #station_max_name = station_all.iloc[wind_max,7]
        #station_max_data = self.station_data[station_max_name]
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:  
            data= grouped_IIiii.get_group(i)
            if data['fFy'].max()>187:
                data_single = {}
                data_single['IIiii'] = data['IIiii'].iloc[0]
                data_single['county'] = data['county'].iloc[0]
                data_single['town'] = data['Town'].iloc[0]
                data_single['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['fFy'].max()/10.0]
                data_single['symbol'] = 'path://M10 10L50 10 50 20 20 20 20 40 50 40 50 50 20 50 20 100 10 100 10 10z'
                data_single['symbolRotate'] = data[data['fFy'] == data['fFy'].max()]['dFy'].iloc[0]
                sort_data['IIiii'].append(data['IIiii'].iloc[0])
                sort_data['county'].append(data['county'].iloc[0])
                sort_data['town'].append(data['Town'].iloc[0])
                sort_data['value'].append(data['fFy'].max()/10.0)
                data_wind_list.append(data_single)
        # 对数据进行排序
        max_sort = max(sort_data['value'])
        level_sort = np.linspace(start = 0.0, stop = max_sort, num = 9)
        sort_data = pd.DataFrame(sort_data)    
        sort_data['index'] = sort_data['value'].rank(ascending=0,method='dense')
        sort_out = sort_data.sort_values(by =['value'],ascending = [False]) 
        sort_html = ''
        # <tr><th>排序</th><th>乡镇</th><th>站点</th><th>能见度</th></tr>
        for i in sort_out['index']:
            table_html = '<tr><th>' + str(int(i)) + '</th><th>' + \
                          str(sort_out[sort_out['index']==i]['town'].iloc[0]) + '</th><th>' + \
                          str(sort_out[sort_out['index']==i]['IIiii'].iloc[0]) + '</th><th>' + \
                          str(sort_out[sort_out['index']==i]['value'].iloc[0]) + '</th></tr>'
            sort_html = sort_html + table_html
        return data_wind_list , sort_html
    def view_data(self):
        '''
        1.根据sql语句计算低能见度的分布和排序
        '''
        data_wind_list = []
        station_all = self.station_all
        sort_data = {
            'IIiii':[],
            'county':[],
            'town':[],
            'value':[]
        }
        grouped_IIiii = station_all.groupby('IIiii')
        for i in grouped_IIiii.size().index:  
            data= grouped_IIiii.get_group(i)
            if (data['VV'].min()<500)&(data['VV'].min()>0):
                data_single = {}
                data_single['IIiii'] = data['IIiii'].iloc[0]
                data_single['county'] = data['county'].iloc[0]
                data_single['town'] = data['Town'].iloc[0]
                data_single['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['VV'].min()]
                sort_data['IIiii'].append(data['IIiii'].iloc[0])
                sort_data['county'].append(data['county'].iloc[0])
                sort_data['town'].append(data['Town'].iloc[0])
                sort_data['value'].append(data['VV'].min())
                data_wind_list.append(data_single)
        #对数据进行排序
        max_sort = max(sort_data['value'])
        level_sort = np.linspace(start = 0.0, stop = max_sort, num = 9)
        sort_data = pd.DataFrame(sort_data)    
        sort_data['index'] = sort_data['value'].rank(ascending=1,method='dense')
        sort_out = sort_data.sort_values(by =['value'],ascending = [True]) 
        return data_wind_list , sort_out
    def temp_data(self):
        '''
        1.根据sql语句计算高低温的分布和排序
        '''
        temp_station_list = ['K8719','K8425','K8674']
        data_temp_max = []
        data_temp_min = []
        for i in temp_station_list:
            single_data_max = {}
            single_data_min = {}
            station_name = i
            single_data_max['name'] = i
            single_data_max['value'] = [self.station_data[i]['lon'].iloc[0],self.station_data[i]['lat'].iloc[0],self.station_data[i]['T'].max()/10.0]
            single_data_min['name'] = i
            single_data_min['value'] = [self.station_data[i]['lon'].iloc[0],self.station_data[i]['lat'].iloc[0],self.station_data[i]['T'].min()/10.0]
            data_temp_max.append(single_data_max)
            data_temp_min.append(single_data_min)
        return data_temp_max,data_temp_min   
    def pre_day(self,date,get_1st,get_2st):
        '''
        日报的响应时间、触发后统计对应天的灾情
        '2022-04-14', 200,'高温 浓雾', '降水 大风', 
        '''
        #date = ['2022-04-14', 200,'低温 浓雾', '降水 大风', 200]
        # 计算时间
        today = datetime.datetime.strptime(date,'%Y-%m-%d')
        start_time = str(today+datetime.timedelta(days = -1))[0:10] + ' 20:00'
        end_time = str(today+datetime.timedelta(days = 1))[0:10] + ' 08:00'
        data_all = self.station_all
        data_time = data_all[(data_all['tTime'] >= start_time) & (data_all['tTime'] <= end_time)]
        
        grouped_IIiii = data_time.groupby('IIiii')
        # 所需数据库
        pre_list = []
        wind_list = []
        view_list = []
        tmax_list = []
        tmin_list = []
        
        for i in grouped_IIiii.size().index:  
            data = grouped_IIiii.get_group(i)
            # print(data)   
            # 低温
            if data[data['T'] > -999]['T'].min() < 30:
                tmin_dir = {}
                tmin_dir['name'] = data['IIiii'].iloc[0]
                tmin_dir['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['T'].tolist(),data['T'].min()]
                tmin_list.append(tmin_dir)
            # 高温   
            if data['T'].max() > 350:
                tmax_dir = {}
                tmax_dir['name'] = data['IIiii'].iloc[0]
                tmax_dir['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['T'].tolist(),data['T'].max()]
                tmax_list.append(tmax_dir)
            # 大风
            if data[data['fFy']>187]['fFy'].max():
                wind_dir = {}
                wind_dir['name'] = data['IIiii'].iloc[0]
                wind_dir['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['fFy'].tolist(),data['fFy'].max()] 
                wind_dir['symbol'] = 'path://M10 10L50 10 50 20 20 20 20 40 50 40 50 50 20 50 20 100 10 100 10 10z'
                wind_dir['symbolRotate'] = data[data['fFy'] == data['fFy'].max()]['dFy'].iloc[0]
                wind_list.append(wind_dir)
            # 能见度
            if data[(data['VV']<500) & (data['VV']>0)]['VV'].min() :
                view_dir = {}
                view_dir['name'] = data['IIiii'].iloc[0]
                view_dir['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['VV'].tolist(),data['VV'].min()] 
                view_list.append(view_dir)                
            # 降水
            if data['RR'].max() > 0:
                pre_dir = {}
                data['RR'].replace(-9999,np.nan,inplace=True)
                pre_dir['name'] = data['IIiii'].iloc[0]
                pre_dir['value'] = [data['lon'].iloc[0],data['lat'].iloc[0],data['RR'].tolist(),data['RR'].sum()] 
                pre_list.append(pre_dir)        
        print('计算指定日期的数据',start_time,end_time,pre_list)
    def text_data(self):
        '''
            用来处理风雨情统计数据
            面雨量
            
        
        '''
        data_all = self.station_all
        data_county = {}
        # sql 语句查询面雨量
        grouped_county = data_all.groupby('county')
        county = []
        for i in grouped_county.size().index:
            data = grouped_county.get_group(i)
            group_time = data.groupby('tTime')
            print(data['county'].iloc[0])
            rain = []
            for j in group_time.size().index:
                data_time = group_time.get_group(j)
                data_time['RR'].replace(-9999,np.nan,inplace=True) 
                rain.append(data_time['RR'].sum()) 
            county.append(np.mean(rain))
        print(county)
        
        # 单站遍历
        station_list = []
        Rsum_list = []
        Rmax_list = []
        Vmin_list = []
        Wmax_list = []
        
        grouped_IIiii = data_all.groupby('IIiii')
        for i in grouped_IIiii.size().index: 
            data = grouped_IIiii.get_group(i)
            data['RR'].replace(-9999,np.nan,inplace=True)
            data['T'].replace(-9999,np.nan,inplace=True)
            data['fFy'].replace(-9999,np.nan,inplace=True)
            data['VV'].replace(-9999,np.nan,inplace=True)
            station_list.append(data['IIiii'].iloc[0])
            Rsum_list.append(data['RR'].sum())
            Rmax_list.append(data['RR'].max())
            Vmin_list.append(data['VV'].max())
            Wmax_list.append(data['fFy'].max())
        pd_Rsum =  pd.DataFrame(
            data= {
                'IIiii':station_list,
                'Rsum':Rsum_list,
                'Rmax':Rmax_list,
                'Vmin':Vmin_list,
                'Wmax':Wmax_list
            }
        ) 
#         print('用来处理风雨情统计数据',pd_Rsum.sort_values('Rsum',ascending =False ).head(5)['IIiii'].tolist())        
        print('用来处理风雨情统计数据',pd_Rsum.sort_values('Rsum',ascending =False ).head(5)['IIiii'].tolist())

    

# start = '2022-01-25 20:00'
# end = '2022-02-10 06:00'


start = '2022-01-25'
end = '2022-02-10'
date = '2022-02-08'
get_1st,get_2st = '低温 浓雾','降水 大风'
sql_worker = zdz_data(start,end)
# data_wind_list , sort_out = sql_worker.wind_data( )
# a = sql_worker.read_csv()
# sql_worker.pre_day(date,get_1st,get_2st)

sql_worker.text_data()


# b = sql_worker.return_daylist()

三门县
临海市
仙居县
天台县
椒江区
温岭市
玉环县
玉环市
路桥区
黄岩区
[103.98961038961039, 139.33246753246752, 126.48831168831168, 132.99740259740258, 31.56103896103896, 70.8103896103896, 1.2114882506527416, 26.677922077922076, 37.922077922077925, 89.49090909090908]
用来处理风雨情统计数据 ['K8914', 'K8785', 'K8709', 'K8774', 'K8924']
CPU times: user 9.02 s, sys: 153 ms, total: 9.17 s
Wall time: 9.13 s


In [20]:
b

,IIiii,county,town,value,index
9,K8719,天台县,坦头镇,57,1.0
3,K8425,温岭市,石塘镇,68,2.0
4,K8502,玉环市,干江镇,102,3.0
1,K8315,路桥区,金清镇,130,4.0
10,K8841,三门县,健跳镇,139,5.0
8,K8674,临海市,永丰镇,158,6.0
12,K8951,仙居县,横溪镇,167,7.0
11,K8929,仙居县,白塔镇,176,8.0
2,K8320,路桥区,金清镇,193,9.0
6,K8513,玉环市,鸡山乡,268,10.0


In [11]:
a

[{'IIiii': 'K8210',
  'county': '黄岩区',
  'town': '上垟乡',
  'value': [121.008268, 28.515366, 393]},
 {'IIiii': 'K8315',
  'county': '路桥区',
  'town': ' 金清镇',
  'value': [121.5944, 28.56139, 130]},
 {'IIiii': 'K8320',
  'county': '路桥区',
  'town': ' 金清镇',
  'value': [121.6653, 28.45833, 193]},
 {'IIiii': 'K8425',
  'county': '温岭市',
  'town': '石塘镇',
  'value': [121.6356, 28.23111, 68]},
 {'IIiii': 'K8502',
  'county': '玉环市',
  'town': '干江镇',
  'value': [121.3717, 28.16639, 102]},
 {'IIiii': 'K8510',
  'county': '玉环市',
  'town': '龙溪镇',
  'value': [121.3092, 28.19611, 495]},
 {'IIiii': 'K8513',
  'county': '玉环市',
  'town': '鸡山乡',
  'value': [121.36417, 28.10611, 268]},
 {'IIiii': 'K8627',
  'county': '临海市',
  'town': '上盘镇',
  'value': [121.7594, 28.69111, 328]},
 {'IIiii': 'K8674',
  'county': '临海市',
  'town': '永丰镇',
  'value': [121.0464, 28.86972, 158]},
 {'IIiii': 'K8719',
  'county': '天台县',
  'town': '坦头镇',
  'value': [121.1022, 29.12, 57]},
 {'IIiii': 'K8841',
  'county': '三门县',
  'town': 

In [3]:
a

,Unnamed: 0,StationName,county,Town,lat,lon,IIiii,tTime,dFy,fFy,T,Tx,Tn,VV,RR
0,0,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 20:00:00,50,32,101,102,101,5149,0
1,1,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 20:00:00,26,134,92,94,92,4605,0
2,2,坎门,玉环市,坎门街道,28.07750,121.27083,58667,2022-01-25 20:00:00,20,78,93,93,92,5721,0
3,3,洪家,椒江区,洪家街道,28.61806,121.41639,58665,2022-01-25 21:00:00,61,27,99,101,99,4309,0
4,4,大陈,椒江区,大陈镇,28.44600,121.90333,58666,2022-01-25 21:00:00,31,113,92,93,92,5122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170994,170994,柯思,仙居县,田市镇,28.66389,120.67080,K8908,2022-02-10 06:00:00,0,0,54,55,53,-9999,0
170995,170995,淡竹,仙居县,淡竹乡,28.67194,120.57060,K8911,2022-02-10 06:00:00,265,3,53,54,53,-9999,0
170996,170996,大陈村雪深,仙居县,埠头镇,28.85028,120.51170,K8931,2022-02-10 06:00:00,-9999,-9999,-9999,-9999,-9999,-9999,0
170997,170997,西陆,仙居县,下各镇,28.82278,120.84830,K8950,2022-02-10 06:00:00,130,8,56,57,56,-9999,0


In [13]:
import numpy as np
import pandas as pd
df = pd.DataFrame({
    'id':['x', 'y', 'z'],
    'a':[3,4,2],
    'b':[0,5,7],
    'c':[1,0,1]}
)
df['最高分科目']=df.iloc[:,1].idxmax(axis=0)
df

,id,a,b,c,最高分科目
0,x,3,0,1,1
1,y,4,5,0,1
2,z,2,7,1,1
